# README: How to train a model on a dataset of Bach Chorales

In [1]:
import tensorflow as tf

from music_generator.model import MusicModel
from music_generator.serializers.discrete_time_serializer import DiscreteTimeMidiSerializer
import music_generator.utilities.sequence_utils as sequence_utils
import music_generator.utilities.utils as utils

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0-dev20200327


# Check if CUDA and GPU are working

In [2]:
for message in utils.check_cuda_and_gpu():
    print(message)

CUDA and GPU Available.


# Create model

The default model architecture and hyperparameters are used.

In [3]:
description = 'bach_chorales'

model = MusicModel(
    ckpt_dir='./training_checkpoints/{}'.format(description),
    log_dir='./logs/{}'.format(description))

model.compile()

model.summary()

Model: "music_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         45568     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1312768   
_________________________________________________________________
dropout (Dropout)            (None, None, 512)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 512)         2048      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 512)         

# Create Dataset

A folder of MIDI files are serialized into event sequences.

The sequences are windowed and the last event split off to use as a label for the model to train on - predicting the next event from the given sequence.

A training and validation dataset are created from the sequences and labels - the validation dataset is used in a callback to reduce the learning rate when validation loss plateaus.

In [4]:
data_path = './training_data/bach_chorales/'

serializer = DiscreteTimeMidiSerializer()

sequences = serializer.serialize_folder(data_path)
sequences, labels = sequence_utils.window(sequences)

dataset_train, dataset_val = sequence_utils.make_tf_datasets(sequences, labels)

Training Sequences: 412325
Validation Sequences: 21702


# Train Model on Dataset

Progress can also be monitored via TensorBoard.

The learning rate of the optimizer will be reduced when validation loss stalls.

The checkpoint with the best training loss will be saved.

In [5]:
epochs = 100

try:
    history = model.fit(dataset_train,
                    validation_data=dataset_val,
                    epochs=epochs,
                    verbose=1)
except KeyboardInterrupt as e:
    print('\nStopping training...')

Epoch 1/100
3221/3221 [==============================] - 480s 149ms/step - loss: 1.1480 - sparse_categorical_accuracy: 0.6765 - val_loss: 0.8154 - val_sparse_categorical_accuracy: 0.7439 - lr: 3.0000e-04
Epoch 2/100
3221/3221 [==============================] - 496s 154ms/step - loss: 0.6871 - sparse_categorical_accuracy: 0.7795 - val_loss: 0.6062 - val_sparse_categorical_accuracy: 0.8027 - lr: 3.0000e-04
Epoch 3/100
3221/3221 [==============================] - 497s 154ms/step - loss: 0.5649 - sparse_categorical_accuracy: 0.8134 - val_loss: 0.5539 - val_sparse_categorical_accuracy: 0.8192 - lr: 3.0000e-04
Epoch 4/100
3221/3221 [==============================] - 498s 155ms/step - loss: 0.4848 - sparse_categorical_accuracy: 0.8363 - val_loss: 0.5052 - val_sparse_categorical_accuracy: 0.8353 - lr: 3.0000e-04
Epoch 5/100
3221/3221 [==============================] - 501s 156ms/step - loss: 0.4232 - sparse_categorical_accuracy: 0.8553 - val_loss: 0.4885 - val_sparse_categorical_accuracy: 0.84